In [8]:
import cv2
import imutils
import numpy as np
bg=None

In [9]:
def run_avg(image,aweight):
    global bg
    if bg is None:
        bg=image.copy().astype("float")
        return
    cv2.accumulateWeighted(image,bg,aweight)

In [10]:
def segment (image,threshold=25):
    global bg
    diff=cv2.absdiff(bg.astype("uint8"),image)
    thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)[1]
    cnts,hierachy=cv2.findContours(thresholded,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    if len(cnts)==0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return(thresholded,segmented)

In [11]:
from sklearn.metrics import pairwise
def countfingers(thresholded,segmented):
    chull=cv2.convexHull(segmented)
    extreme_top=tuple(chull[chull[:,:,1].argmin()][0])
    extreme_bottom=tuple(chull[chull[:,:,1].argmax()][0])
    extreme_left=tuple(chull[chull[:,:,0].argmin()][0])
    extreme_right=tuple(chull[chull[:,:,0].argmax()][0])
    cX = int((extreme_left[0] + extreme_right[0]) / 2)
    cY = int((extreme_top[1] + extreme_bottom[1]) / 2)
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[extreme_left, extreme_right, extreme_top, extreme_bottom])[0]
    maximum_distance = distance[distance.argmax()]
    radius = int(0.8 * maximum_distance)
    circumference = (2 * np.pi * radius)
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    cv2.circle(circular_roi, (cX, cY), radius, 255, 1)
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    cnts,hierachy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    count = 0
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1
    return count

In [13]:
if __name__=='__main__':
    aweight=0.5
    top,right,bottom,left=10,350,225,590
    num_frames=0
    num_fingers=0
    min_YCrCb = np.array([0,133,77],np.uint8)
    max_YCrCb = np.array([255,173,127],np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.namedWindow('Camera Output')
    videoFrame = cv2.VideoCapture(0)
    while(True):
        readSucsess, sourceImage = videoFrame.read()
        skin = cv2.flip(sourceImage, 1)
#         imageYCrCb = cv2.cvtColor(sourceImage,cv2.COLOR_BGR2YCR_CB)
#         skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
#         skinRegion = cv2.GaussianBlur(skinRegion, (3, 3), 0)
#         skin = cv2.bitwise_and(sourceImage, sourceImage, mask = skinRegion)
        clone=skin.copy()
        (height,width)=skin.shape[:2]
        roi=skin[top:bottom,right:left]
        gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
        gray=cv2.GaussianBlur(gray,(7,7),0)
        if num_frames<30:
            run_avg(gray,aweight)
        else:
            hand=segment(gray)
            if hand is not None:
                (thresholded,segmented)=hand
                cv2.drawContours(clone,[segmented+(right,top)],-1,(0,0,255))
                fingers = countfingers(thresholded, segmented)
                cv2.putText(clone, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                cv2.imshow("Thesholded", thresholded)
        cv2.rectangle(clone,(left,top),(right,bottom),(0,255,0),2)
        num_frames+=1
        cv2.imshow("Video Feed",clone)
        keypress=cv2.waitKey(1) & 0xFF
        if keypress== ord('q'):
            break
videoFrame.release()
cv2.destroyAllWindows()

In [1]:
#skin detection
import cv2
import numpy
min_YCrCb = numpy.array([0,133,77],numpy.uint8)
max_YCrCb = numpy.array([255,173,127],numpy.uint8)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.namedWindow('Camera Output')
videoFrame = cv2.VideoCapture(0)
while True: 
    readSucsess, sourceImage = videoFrame.read()
    sourceImage = cv2.flip(sourceImage, 1)
    imageYCrCb = cv2.cvtColor(sourceImage,cv2.COLOR_BGR2YCR_CB)
    skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
    skinRegion = cv2.GaussianBlur(skinRegion, (3, 3), 0)
    skin = cv2.bitwise_and(sourceImage, sourceImage, mask = skinRegion)
    cv2.imshow('Camera Output',skin)
    keypress=cv2.waitKey(1) & 0xFF
    if keypress== ord('q'):
        break
cv2.destroyWindow('Camera Output')
videoFrame.release()